## Лабораторная работа №3

### Импорты

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from sklearn.utils import shuffle
from tensorflow.keras import optimizers

import numpy as np

from datetime import datetime

### Загрузка данных

In [2]:
BATCH = 128
datagen = ImageDataGenerator(rescale=1./255)
    
train_data = datagen.flow_from_directory('../../intel-image-classification/seg_train/seg_train/',
                                        target_size=(150, 150),
                                        batch_size=BATCH,
                                        class_mode='categorical',
                                        shuffle=True)

test_data = datagen.flow_from_directory('../../intel-image-classification/seg_test/seg_test/',
                                        target_size=(150, 150),
                                        batch_size=BATCH,
                                        class_mode='categorical',
                                        shuffle=True)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [3]:
train_data.class_indices

{'buildings': 0,
 'forest': 1,
 'glacier': 2,
 'mountain': 3,
 'sea': 4,
 'street': 5}

In [8]:
train_data.labels.shape

(14034,)

### Конфигурация №1

In [9]:
model1 = Sequential([
    layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3, 3), activation = 'relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model1.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [10]:
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=5, validation_steps=len(test_data), validation_data=test_data)

Epoch 1/5
110/110 [==============================] - 219s 2s/step - loss: 1.3370 - accuracy: 0.5282 - val_loss: 0.8883 - val_accuracy: 0.6410
Epoch 2/5
110/110 [==============================] - 213s 2s/step - loss: 0.7621 - accuracy: 0.7168 - val_loss: 0.6997 - val_accuracy: 0.7517
Epoch 3/5
110/110 [==============================] - 213s 2s/step - loss: 0.6054 - accuracy: 0.7815 - val_loss: 0.6725 - val_accuracy: 0.7583
Epoch 4/5
110/110 [==============================] - 213s 2s/step - loss: 0.4910 - accuracy: 0.8252 - val_loss: 0.7370 - val_accuracy: 0.7277
Epoch 5/5
110/110 [==============================] - 214s 2s/step - loss: 0.3790 - accuracy: 0.8707 - val_loss: 0.7131 - val_accuracy: 0.7470


### Конфигурация №2

In [4]:
model = Sequential([
    layers.Conv2D(128, (3, 3), activation = 'tanh', input_shape = (150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation = 'tanh'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3, 3), activation = 'tanh'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [5]:
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=5, validation_steps=len(test_data), validation_data=test_data)

Epoch 1/5


ResourceExhaustedError: OOM when allocating tensor with shape[128,148,148,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv2D]

### Конфигурация №3

In [6]:
model = Sequential([
    layers.Conv2D(256, (3, 3), activation = 'relu', input_shape = (150, 150, 3)),
    layers.MaxPooling2D(5, 5),
    layers.Conv2D(128, (3, 3), activation = 'relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation = 'relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3, 3), activation = 'relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=10, validation_steps=len(test_data), validation_data=test_data)

Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[128,148,148,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv2D]

### Конфигурация №4

In [18]:
model = Sequential([
    layers.Conv2D(256, (3, 3), activation = 'relu', input_shape = (150, 150, 3)),
    layers.MaxPooling2D(5, 5),
    layers.Conv2D(180, (3, 3), activation = 'relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation = 'relu'),
    layers.Conv2D(64, (3, 3), activation = 'relu'),
    layers.Conv2D(32, (3, 3), activation = 'relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(180,activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=15, validation_steps=len(test_data), validation_data=test_data)

Epoch 1/15


ResourceExhaustedError: OOM when allocating tensor with shape[128,148,148,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv2D]

### Конфигурация №5

In [21]:
model = Sequential([
    layers.Conv2D(256, (3, 3), activation = 'relu', input_shape = (150, 150, 3)),
    layers.Conv2D(180, (3, 3), activation = 'relu'),
    layers.MaxPooling2D(5, 5),
    layers.Conv2D(180, (3, 3), activation = 'relu'),
    layers.Conv2D(128, (3, 3), activation = 'relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation = 'relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3, 3), activation = 'relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(180,activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(50,activation='relu'),
    layers.Dropout(rate=0.5),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=25, validation_steps=len(test_data), validation_data=test_data)

Epoch 1/25


ResourceExhaustedError: OOM when allocating tensor with shape[128,148,148,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv2D]